In [12]:
# Standard libraries
import numpy as np

# Append base directory
import os,sys,inspect
rootname = "neuronal-sequence-test"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

# User libraries
from mesostat.utils.qt_helper import gui_fnames, gui_fpath

from src.lib.data_db import BehaviouralNeuronalDatabase
from src.lib.metric_wrapper import metric_by_interval, metric_by_phase
from src.lib.extra_metrics import num_non_zero_std, num_sample

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/neuronal-sequence-test
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
#params['root_path_data']  = gui_fpath("Path to data files", "./")
params['root_path_dff'] = '/media/alyosha/Data/TE_data/mariadata/dff/'
params['root_path_deconv'] = '/media/alyosha/Data/TE_data/mariadata/deconv/'

In [14]:
dataDB = BehaviouralNeuronalDatabase(params)
dataDB.read_neuro_files()
dataDB.read_behavior_files()

IntProgress(value=0, description='Read DFF Data:', max=27)

IntProgress(value=0, description='Read DECONV Data:', max=27)

IntProgress(value=0, description='Read Neuro Data:', max=27)

No trials found for Trial_LWhole_Mistake skipping
No trials found for Trial_RWhole_Mistake skipping
No trials found for Trial_LWhole_Mistake skipping


# TESTS

Sweeps:
* Mice: All, exact
* Performance: Correct, Mistake, All
* Direction: L, R, All
* Projectors: Intervals, Phases
* TrgDimensions: "", "p", "pr"
* multiplexKey: None, mousename, session

In [61]:
miceSweep = [None] + list(dataDB.mice)
perfSweep = ["Correct", "Mistake"]
dirSweep = ["L", "R", None]
trgDimSweep = ["", "p"]  # TODO: "pr"
multiplexKeySweep = [None, "mousename", "session"]
dataTypeSweep = ["raw", "deconv"]

In [62]:
metricName = "mean"

def append(d, k, v):
    if v is not None:
        d[k] = v
        
def test(txt, a, b):
    if not np.all(a==b):
        raise ValueError("For", txt, "expected", b, "got", a)
    else:
        print("For", txt, "test passed")

for mousename in miceSweep:
    nMice = len(dataDB.mice) if mousename is None else 1
    for dataType in dataTypeSweep:
        nSession = dataDB.get_nsession(mousename, dataType)

        for performance in perfSweep:
            nIntervals = len(dataDB.metaDataFrames['interval_maps'][performance]) - 1
            nPhases = len(dataDB.phases[performance])

            for direction in dirSweep:
                trgDimSweepEff = trgDimSweep if mousename is not None else [""]

                for trgDimOrd in trgDimSweepEff:
                    multiplexKeySweepEff = multiplexKeySweep if mousename is None else [None]
                    for multiplexKey in multiplexKeySweepEff:
                        queryThis = {}
                        append(queryThis, "datatype", dataType)
                        append(queryThis, "mousename", mousename)
                        append(queryThis, "performance", performance)
                        append(queryThis, "direction", direction)

                        settings = {"zscoreChannel" : False, "serial" : True, "metricSettings" : None}

                        display("Testing", {**queryThis, **settings, "dimOrdTrg" : trgDimOrd, "multiplexKey" : multiplexKey})

                        rezInterv = metric_by_interval(dataDB, queryThis, metricName, trgDimOrd, settings, multiplexKey=multiplexKey)
                        rezPhase = metric_by_phase(dataDB, queryThis, metricName, trgDimOrd, settings, multiplexKey=multiplexKey)

                        testIntervShape = (nIntervals, )
                        testPhaseShape = (nPhases, )

                        if multiplexKey is not None:
                            if multiplexKey == "mousename":
                                testIntervShape += (nMice, )
                                testPhaseShape += (nMice, )
                            elif multiplexKey == "session":
                                testIntervShape += (nSession, )
                                testPhaseShape += (nSession, )

                            #if mousename is not None:

                        if "p" in trgDimOrd:
                            nChannels = dataDB.get_nchannel(mousename, dataType)
                            testIntervShape += (nChannels, )
                            testPhaseShape += (nChannels, )

                        test("Interval", rezInterv.shape, testIntervShape)
                        test("Phase", rezPhase.shape, testPhaseShape)

'Testing'

{'datatype': 'raw',
 'performance': 'Correct',
 'direction': 'L',
 'zscoreChannel': False,
 'serial': True,
 'metricSettings': None,
 'dimOrdTrg': '',
 'multiplexKey': None}

For Interval test passed
For Phase test passed


'Testing'

{'datatype': 'raw',
 'performance': 'Correct',
 'direction': 'L',
 'zscoreChannel': False,
 'serial': True,
 'metricSettings': None,
 'dimOrdTrg': '',
 'multiplexKey': 'mousename'}

...

{'datatype': 'deconv',
 'mousename': 'm061',
 'performance': 'Mistake',
 'zscoreChannel': False,
 'serial': True,
 'metricSettings': None,
 'dimOrdTrg': '',
 'multiplexKey': None}

For Interval test passed
For Phase test passed


'Testing'

{'datatype': 'deconv',
 'mousename': 'm061',
 'performance': 'Mistake',
 'zscoreChannel': False,
 'serial': True,
 'metricSettings': None,
 'dimOrdTrg': 'p',
 'multiplexKey': None}

For Interval test passed
For Phase test passed
